In [1]:
import pandas as pd
import numpy as np
from typing import *
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch
import sklearn
from sentence_transformers import SentenceTransformer

In [2]:
import sklearn.cluster
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    # print(last_hidden.shape)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [3]:
# tokenizer = AutoTokenizer.from_pretrained(r"Embedding_Model/gte_base")
# model = AutoModel.from_pretrained(r"Embedding_Model/gte_base")
model: SentenceTransformer = SentenceTransformer('thenlper/gte-base')

In [4]:
# val1=tokenizer.save_pretrained(r"Embedding_Model/gte_large")
# val2= model.save_pretrained(r"Embedding_Model/gte_large/")
# model.save(r"Embedding_Model/gte_base_sentence")

# att= tokenizer(temp_project[0], padding=True, truncation=True, return_tensors="pt")
# out_test = model(**att)
# print(out_test.last_hidden_state.shape, type(out_test), "\n", len(temp_project[0])\
#       , sum([len(val) for val in temp_project[0]]), max([len(val) for val in temp_project[0]]))
# # print(model.config)
# embedds = average_pool(out_test.last_hidden_state, att["attention_mask"])
# print(embedds.shape)

In [5]:
data_input = pd.read_csv(r"Generated_Result/10k.csv", sep=";")

In [6]:
data_input.fillna("None", inplace=True)
data_input.head()

C:\Users\jqi22\AppData\Local\Temp\ipykernel_19784\922194924.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_input.fillna("None", inplace=True)


,project,risk,stakeholder,idx
0,Geological CO2 sequestration,Mitigation of anthropogenic carbon dioxide emi...,None,0.0
1,development and validation of a discrete event...,None,commercial vehicles passing through the area,1.0
2,Urban renaissance project in the area of Gazi,Pollution from the industrial use of Gazi|whic...,Residents of Gazi|environmental activists|loca...,2.0
3,simulation of changes in live carbon stocks in...,droughts,forest ecosystem|climate change,4.0
4,Ecological enhancement of coastal and marine i...,degraded or reduced habitat for native species,ecosystem and habitat value|this paper present...,5.0


In [7]:
stupid: Dict[str, List] = {}
for col in ["project", "risk", "stakeholder"]:
    temp = data_input.loc[:, col]
    blyat = []
    for tep_val in temp.values.tolist():
        if tep_val[0] != tep_val[0] or tep_val=="None": continue
        tep_val = tep_val.split("|")
        blyat = [*blyat, *tep_val]
    stupid[col] = blyat

# stupid_copy = {col: [val for val in data_input[col].str.split("|", expand=True).stack().dropna()]\
#            for col in ["Project", "Risk", "Stakeholder"]}
print(len(stupid["project"]), len(stupid["risk"]), len(stupid["stakeholder"]))

11667 14328 16924


In [8]:
temp_project = [stupid["risk"][val: val+5] for val in range(0, len(stupid["risk"]), 5)]

In [9]:
# stupid["stakeholder"][:5]

In [10]:
import math
class Node:
  def __init__(self, name, embeddings):
    self.name: str =name
    self.embeddings: np.ndarray = embeddings

  def __call__(self):
    return self.embeddings

def embedding_value(col, max_lim=1000, threshold=200):
    lens = max_lim if max_lim > threshold else len(stupid[col])
    output_project = []
    for item in range(0, math.ceil(lens/threshold)):
        centre = stupid[col][item*threshold:(item+1)*threshold if (item+1)*threshold<lens-1 else lens-1]
        print(item, len(centre))
        # Tokenize the input texts
        res = model.encode(centre, batch_size=len(centre), convert_to_numpy=1) # return a numpy vector in threhold * embedding dim shape
        output_project.extend([Node(*new_one) for new_one in zip(centre, [*res])])
    return output_project # use multi-processing next time

In [11]:
columns = data_input.columns.to_list()
print(columns)
# projectRes = embedding_value(col=columns[0], max_lim=1000, threshold=200)
# len(projectRes)

['project', 'risk', 'stakeholder', 'idx']


In [17]:
from importlib import reload
import word_embedding_acc as weacc
reload(weacc)

'risk'

In [20]:
%%time
import concurrent.futures
import gc

counter = 0
def main():
    dataset = pd.DataFrame(columns=["name", *[f"embedding_{val}" for val in range(768)]])
    projectRes, newShape = [], 0
    with concurrent.futures.ThreadPoolExecutor() as executor:
        projectRes = executor.submit(embedding_value, col=columns[1], max_lim=10, threshold=200).result()
    newShape = len(projectRes)
    for items in range(newShape):
        pidx: Node = projectRes[items]
        data = [pidx.name, *pidx.embeddings]
        dataset.loc[items, ] = pd.Series(data, index=dataset.columns)
    return dataset

if __name__ == "__main__":
    dataset = weacc.acc(columns, 1, stupid, model)
    gc.collect()
    print(dataset.head(3), dataset.shape)

                                                name embedding_0 embedding_1  \
0  Mitigation of anthropogenic carbon dioxide emi...    0.003574   -0.003208   
1  injection capacity and safety of the storage s...   -0.021322   -0.006011   
2                         fluid flow in porous media   -0.000565    0.008649   

  embedding_2 embedding_3 embedding_4 embedding_5 embedding_6 embedding_7  \
0    0.018408    0.008215    0.070227    0.022868    0.015587    0.033817   
1   -0.011679     0.03773    0.094119    0.019343    0.034572    0.022691   
2    0.016875    0.008861    0.057814    0.023462    0.031459     0.04278   

  embedding_8  ... embedding_758 embedding_759 embedding_760 embedding_761  \
0   -0.020889  ...      0.041077      -0.01497     -0.035081     -0.023601   
1   -0.031523  ...      0.004187     -0.016691      0.010008     -0.041102   
2   -0.009724  ...      0.038016      -0.00126     -0.019946     -0.059347   

  embedding_762 embedding_763 embedding_764 embedding_765

In [21]:
gc.collect()
dataset.to_csv(r"Generated_Result/10k_risk_embedding.csv", sep=";", index=False)

In [40]:
import gc
gc.collect()

In [21]:
columns = data_input.columns.to_list()
temp = ["risk", "project"]

threshold = 200
lens = len(stupid[col])
new_item_list = []
for item in range(0, math.ceil(lens/threshold)):
    new_item_list.append(stupid[col][:(item+1)*threshold] if (item+1)*threshold<lens-1 else lens-1)
    
for col in temp:
    embedding_value(col=col, threshold=200)

In [14]:
# beside stakeholder, rest of them shall be reloaded again
torch.save(torch.cat(output_project, axis=0), "Embedding_Model/gte_large_embedding/three_ouput1_risk.pt")

In [14]:
# import gc
# gc.collect()
torch.cuda.empty_cache()

In [13]:
full_emd, atten_mask = outputs.last_hidden_state, batch_dict['attention_mask']

In [14]:
print(full_emd.shape, atten_mask.shape, embeddings.shape)

torch.Size([4, 6, 768]) torch.Size([4, 6]) torch.Size([4, 768])


In [13]:
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())

[[69.65817260742188, 88.03556060791016, 68.79690551757812]]


In [1]:
import pandas as pd

In [7]:
d1 = pd.read_csv(r"Generated_Result/1k_fp.csv", sep=';')
d1.to_excel(r"Generated_Result/new_split_quota.xlsx", index=False)